In [ ]:
#| echo: false
#| output: false
Pkg.activate(@__DIR__)
Pkg.instantiate()

## Overview

This tutorial will give some examples of using `Turing.jl` and Markov Chain Monte Carlo to sample from posterior distributions.

## Setup


In [ ]:
using Turing
using Distributions
using Plots
using LaTeXStrings
using StatsPlots
using StatsBase
using Optim
using Random
using DataFrames
using Downloads
using CSV

As this tutorial involves random number generation, we will set a random seed to ensure reproducibility.


In [ ]:
Random.seed!(1);

## Fitting A Linear Regression Model

Let's start with a simple example: fitting a linear regression model to simulated data.

::: {.callout-tip}
## Positive Control Tests

Simulating data with a known data-generating process and then trying to obtain the parameters for that process is an important step in any workflow.
:::

### Simulating Data

The data-generating process for this example will be:
$$
\begin{gather}
y = 5 + 4x + \varepsilon \\
\varepsilon \sim \text{Normal}(0, 10),
\end{gather}
$$
where $\varepsilon$ is so-called "white noise", which adds stochasticity to the data set. The generated dataset is shown in @fig-scatter-regression.


In [ ]:
#| column: margin
#| label: fig-scatter-regression
#| fig-cap: Scatterplot of our generated data.

# create trend for data
x = rand(Uniform(0, 20), 10)
y = 5 .+ 4 * x
# sample and add noise
ε = rand(Normal(0, 10), 10)
y .+= ε

scatter(x, y, label="Data", xlabel=L"$x$", ylabel=L"$y$", markersize=10, tickfontsize=14, guidefontsize=16, legendfontsize=16)

### Model Specification

The statistical model for a standard linear regression problem is
$$
\begin{gather}
y = a + bx + \varepsilon \\
\varepsilon \sim \text{Normal}(0, \sigma^2).
\end{gather}
$$

Rearranging, we can rewrite the likelihood function as:
$$y \sim \text{Normal}(\mu, \sigma^2),$$
where $\mu = a + bx$. This means that we have three parameters to fit: $a$, $b$, and $\sigma^2$.

Next, we need to select priors on our parameters. We'll use relatively generic distributions to avoid using the information we have (since we generated the data ourselves), but in practice, we'd want to use any relevant information that we had from our knowledge of the problem. Let's use relatively diffuse normal distributions for the trend parameters $a$ and $b$ and a half-normal distribution (a normal distribution truncated at 0, to only allow positive values) for the variance $\sigma^2$, as recommended by [Gelman (2006)](https://doi.org/10.1214/06-BA117A).

$$
\begin{gather}
a \sim \text{Normal(0, 10)} \\
b \sim \text{Normal(0, 10)} \\
\sigma^2 \sim \text{Half-Normal}(0, 25)
\end{gather}
$$

### Using Turing

#### Coding the Model

`Turing.jl` uses the `@model` macro to specify the model function. We'll follow the setup in the [Turing documentation](https://turinglang.org/dev/tutorials/05-linear-regression).

To specify distributions on parameters (and the data, which can be thought of as uncertain parameters in Bayesian statistics), use a tilde `~`, and use equals `=` for transformations (which we don't have in this case).


In [ ]:
@model function linear_regression(x, y)
    # set priors
    σ² ~ truncated(Normal(0, 25); lower=0)
    a ~ Normal(0, 10)
    b ~ Normal(0, 10)

    # we can specify the likelihood with a loop, as our data is i.i.d.
    # we could also rewrite this using linear algebra, which might be more efficient for large and/or complex models or datasets, but this will be more readable in this simple case.
    for i = 1:length(y)
        # compute the mean value for the data point
        μ = a + b * x[i]
        y[i] ~ Normal(μ, σ²)
    end
end

#### Fitting The Model

Now we can call the sampler to draw from the posterior. We'll use the [No-U-Turn sampler](http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf), which is a Hamiltonian Monte Carlo algorithm (a different category of MCMC sampler than the Metropolis-Hastings algorithm discussed in class). We'll also use 4 chains so we can test that the chains are well-mixed, and each chain will be run for 5,000 iterations^[Hamiltonian Monte Carlo samplers often need to be run for fewer iterations than Metropolis-Hastings samplers, as the exploratory step uses information about the gradient of the statistical model, versus the random walk of Metropolis-Hastings. The disadvantage is that this gradient information must be available, which is not always the case for simulation models.]


In [ ]:
chain = let
    model = linear_regression(x, y) # create the model object with the data
    sampler = NUTS()
    n_per_chain = 5000
    nchains = 4
    # call the sampler and drop the "burn-in/warm-up" portion
    sample(model, sampler, MCMCThreads(), n_per_chain, nchains, drop_warmup=true);
end

@show chain

How can we interpret the output? The first parts of the summary statistics are straightforward: we get the mean, standard deviation, and Monte Carlo standard error (`mcse`) of each parameter. We also get information about the effective sample size (ESS)^[The ESS reflects the efficiency of the sampler: this is an estimate of the equivalent number of independent samples; the more correlated the samples, the lower the ESS.] and $\hat{R}$, which measures the ratio of within-chain variance and across-chain variance as a check for convergence^[The closer $\hat{R}$ is to 1, the better.].

In this case, we can see that we were able to recover the "true" data-generating values of $\sigma^2 = 10$ and $b = 4$, but $a$ is slightly off (the mean is 3, rather than the data-generating value of 5). This isn't surprising: given the variance of the noise $\sigma^2$, there are many different intercepts which could fit within that spread.

Let's now plot the chains for visual inspection.


In [ ]:
#| label: fig-chains-regression
#| fig-cap: 'Output from the MCMC sampler. Each row corresponds to a different parameter: $\sigma^2$, $a$, and $b$. Each chain is shown in a different color. The left column shows the sampler traceplots, and the right column the resulting posterior distributions.'
#| column: page-inset-right

plot(chain)

We can see from @fig-chains-regression that our chains mixed well and seem to have converged to similar distributions! The traceplots have a "hairy caterpiller" appearance, suggesting relatively little autocorrelation. This aligns with the ESS estimates from the `chain` output, which are higher than the number of drawn samples^[Which is possible with Hamiltonian Monte Carlo due to the efficiency of exploration!]. We can also see how much more uncertainty there is with the intercept $a$, while the slope $b$ is much more constrained, with a 95% credible interval of $(3, 4.8)$ (compared to $(-6.5, 12)$ for $a$).

Another interesting comparison we can make is with the maximum-likelihood estimate (MLE), which we can obtain through optimization.


In [ ]:
#| column: margin

mle_model = linear_regression(x, y); # specify the model object
optimize(mle_model, MLE())

The MLE results have some additional error compared to the MCMC means (the slope and noise variance are further from the data-generating values), but this can be a lot faster, at the expense of getting estimates of uncertainty.

### Model Diagnostics and Posterior Predictive Checks

One advantage of the Bayesian modeling approach here is that we have access to a *generative model*, or a model which we can use to generate datasets. This means that we can now use Monte Carlo simulation, sampling from our posteriors, to look at how uncertainty in the parameter estimates propagates through the model. Let's write a function which gets samples from the MCMC chains and generates datasets.


In [ ]:
function mc_predict_regression(x, chain)
    # the Array(group()) syntax is more general than we need, but will work if we have multiple variables which were sampled as a group, for example multiple regression coefficients.
    a = Array(group(chain, :a))
    b = Array(group(chain, :b))
    σ² = Array(group(chain, :σ²))

    μ = a' .+ x * b'
    y = zeros((length(x), length(a)))
    for i = 1:length(a)
        y[:, i] = rand.(Normal.(μ[:, i], σ²[i]))
    end
    return y
end

Now we can generate a predictive interval and median and compare to the data.


In [ ]:
x_pred = 0:20;
y_pred = mc_predict_regression(x_pred, chain)

Notice the dimension of `y_pred`: we have 20,000 columns, because we have 4 chains with 5,000 samples each. If we had wanted to subsample (which might be necessary if we had hundreds of thousands or millions of samples), we could have done that within `mc_linear_regression` before simulation.


In [ ]:
# get the boundaries for the 95% prediction interval and the median
y_ci_low = quantile.(eachrow(y_pred), 0.025);
y_ci_hi = quantile.(eachrow(y_pred), 0.975);
y_med = quantile.(eachrow(y_pred), 0.5);

Now, let's plot the prediction interval and median, and compare to the original data.


In [ ]:
#| label: fig-prediction-regression
#| fig-cap: Posterior 95% predictive interval and median for the linear regression model. The data is plotted in red for comparison.

# plot prediction interval
plot(x_pred, y_ci_low, fillrange=y_ci_hi, xlabel=L"$x$", ylabel=L"$y$", fillalpha=0.3, fillcolor=:blue, label="95% Prediction Interval", legend=:topleft, linealpha=0)
# plot median line
plot!(x_pred, y_med, color=:blue, label="Prediction Median")
scatter!(x, y, color=:red, label="Data")

From @fig-prediction-regression, it looks like our model might be slightly under-confident, as with 10 data points, we would expect 5% of them (or 1/2 of a "data point") to be outside the 95% prediction interval. It's hard to tell with only 10 data points, though! We could resolve this by tightening our priors, but this depends on how much information we used to specify them in the first place. The goal shouldn't be to hit a specific level of uncertainty, but if there is a sound reason to tighten the priors, we could do so.

Now let's look at the residuals from the posterior median and the data. The partial autocorrelations plotted in @fig-residuals-regression are not fully convincing, as there are large autocorrelation coefficients with long lags, but the dataset is quite small, so it's hard to draw strong conclusions. We won't go further down this rabbit hole as we know our data-generating process involved independent noise, but for a real dataset, we might want to try a model specification with autocorrelated data to compare.


In [ ]:
#| column: margin
#| label: fig-residuals-regression
#| fig-cap: 'Partial autocorrelation function of model residuals, relative to the predictive median.'

y_pred_data = mc_predict_regression(x, chain)
y_med_data = quantile.(eachrow(y_pred_data), 0.5)
residuals = y_med_data .- y
plot(pacf(residuals, 1:4), line=:stem, marker=:circle, legend=:false, grid=:false, linewidth=2, xlabel="Lag", ylabel="Partial Autocorrelation", markersize=8, tickfontsize=14, guidefontsize=16, legendfontsize=16)
hline!([0], linestyle=:dot, color=:red)

## Fitting Extreme Value Models to Tide Gauge Data

Let's now look at an example of fitting an extreme value distribution (namely, a generalized extreme value distribution) to tide gauge data.

First, we'll download and plot the data:


In [ ]:
#| label: fig-tds-scatter
#| fig-cap: 'Total dissolved solids ($mg/L$) vs. discharge ($m^3/s$) for the Cuyahoga River from 1969 to 1973. Data obtained from [Statistical Methods in Water Resources](https://doi.org/10.3133/tm4A3).'

fname = "data/tds.csv" # CHANGE THIS!
url = "https://www.sciencebase.gov/catalog/file/get/5d5af82de4b01d82ce8ed296?f=__disk__b9%2Fb7%2F40%2Fb9b74037d91c45e730ba4c170fe3ffaf9470d360"
if !isfile(fname)
    Downloads.download(url, fname)
end
tds = DataFrame(CSV.File(fname))
tds[!, [:date, :discharge_cms, :tds_mgL]]

p = scatter(tds.discharge_cms, tds.tds_mgL, xlabel=L"Discharge~($m^3/s$)", ylabel=L"Total dissolved solids~($mg/L$)", title="Cuyahoga River, Ohio, 1969–1973", label="Data")

Note at @fig-tds-scatter does not look linear at all. However, it does look roughly like an inverse-exponential plot, which suggests that logarithmically-transforming the $x$-axis might look linear.


In [ ]:
#| label: fig-tds-log-scatter
#| fig-cap: 'The same plot as @fig-tds-scatter, but with the $x$-axis shown on a log scale.'

x_ticks = [1, 2, 5, 10, 20, 50, 100]
plot(p, xaxis=:log, xticks=(x_ticks, string.(x_ticks)))